## Baseline receiver stateless monitoring 

In [ ]:
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages "
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.6,"
    "org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.6 "
    "pyspark-shell"
)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("read_test_straeam") \
        .getOrCreate()

:: loading settings :: url = jar:file:/home/guest/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/guest/.ivy2/cache
The jars for the packages stored in: /home/guest/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f2b9614c-f5e9-4150-93af-126de5ce8f17;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.6 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.6 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in c

In [3]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

KAFKA_BOOTSTRAP_SERVERS = "127.0.0.1:9092"
KAFKA_TOPIC = "sensors"


schema = StructType().add("sensor", StringType())\
                     .add("info", StructType()\
                          .add("timestamp_sent", StringType())\
                          .add("timestamp_received", StringType())\
                          .add("obs", StringType())\
                          .add("drift", StringType()))

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()\
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))\
            .select(col("parsed_value.sensor"), \
                    col("parsed_value.info.timestamp_sent"),\
                    col("parsed_value.info.timestamp_received"),\
                    col("parsed_value.info.obs"),\
                    col("parsed_value.info.drift"))

In [4]:
df.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- timestamp_sent: string (nullable = true)
 |-- timestamp_received: string (nullable = true)
 |-- obs: string (nullable = true)
 |-- drift: string (nullable = true)



In [5]:
watermark=True
WATERMARK_SIZE="1 minute"
WINDOW_SIZE="1 minute"

In [6]:
from pyspark.sql.functions import window, avg, count, desc, stddev, min, max, isnull, col, sum, isnan, unix_timestamp, round
from pyspark.sql.functions import from_unixtime, to_utc_timestamp, date_format

df = df.withColumn("timestamp_received", to_utc_timestamp("timestamp_received", "Europe/Rome"))\
         .withColumn("timestamp_sent", to_utc_timestamp("timestamp_received", "Europe/Rome"))\
            .withColumn("time_diff", (col("timestamp_received").cast("double") - col("timestamp_sent").cast("double")))\


if watermark:
    df = df.withWatermark("timestamp_received", WATERMARK_SIZE)

eventAvg = df.groupBy(window(df.timestamp_received, WINDOW_SIZE), df.sensor)\
                    .agg(
                        count("obs").alias("count_obs"),
                        round(avg("obs"), 3).alias("avg_obs"),
                        round(stddev("obs"), 3).alias("+/-std_obs"),
                        round(max("obs"), 3).alias("max_obs"),
                        round(min("obs"), 3).alias("min_obs"),
                        round(sum((isnull("obs") | isnan("obs")).cast("int")), 3).alias("sum_miss_obs"),
                        round(avg("time_diff"), 3).alias("avg_time_delay")
                        )\
                    .withColumn("window_start", col("window.start")) \
                    .withColumn("window_end", col("window.end"))\
                    .withColumn("day", date_format("window_start", "yyyy-MM-dd")) \
                    .withColumn("hour", date_format("window_start", "HH")) \
                    .drop("window") #.sort(desc("window"))

In [ ]:
eventAvg.writeStream \
        .outputMode("append") \
        .trigger(processingTime="30 seconds") \
        .format("csv") \
        .option("path", "/home/guest/notebooks/data/output/") \
        .option("checkpointLocation", "/home/guest/notebooks/data/checkpoints") \
        .option("header", "true") \
        .partitionBy("day", "hour") \
        .start().awaitTermination()

25/06/26 12:57:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/06/26 12:57:32 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [ ]:
#results = eventAvg.writeStream \
#         .outputMode('complete')\
#         .format("console") \
#         .start() \
#         .awaitTermination()